In [34]:
import h5netcdf

In [1]:
# libraries
import os
import joblib
from osgeo import gdal
import pandas as pd
import numpy as np
import xarray as xr
import glob
import re
import rioxarray
import dask.array
from dask.distributed import Client, progress

In [2]:
client = Client(n_workers=4, threads_per_worker=2)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 4
Total threads: 8,Total memory: 480.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42347,Workers: 4
Dashboard: /proxy/8787/status,Total threads: 8
Started: Just now,Total memory: 480.00 GiB
Comm: tcp://127.0.0.1:41605,Total threads: 2
Dashboard: /proxy/34277/status,Memory: 120.00 GiB
Nanny: tcp://127.0.0.1:41937,


In [3]:
workingPath = "/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/"

In [4]:
# function for getting directory of input data
def get_directories_with_number_and_ending(directory_path, ending):
    directories = []
    if ending:
        pattern = re.compile(r'^\d+.*{}$'.format(re.escape(ending)))  # Match directory names with number at the beginning and specific ending
    else:
        pattern = re.compile(r'^\d+$')  # Match directory names with only numbers

    for entry in os.scandir(directory_path):
        if entry.is_dir():
            directory_name = entry.name
            if pattern.match(directory_name):
                directories.append(directory_name)

    return directories

In [7]:
### define the path of input data, set the reference file for spatial resample
inputData = workingPath+"1input_data/"
# get all the filefolders named as year
year_list = get_directories_with_number_and_ending(workingPath+"1input_data/", "global")
# the input data from year[0] to year[..], based on the process id in sbatch script
year = year_list[4-1]
print(year)

2015global


In [6]:
import geopandas as gpd
from shapely.geometry import box

shapefile_path = "/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/Emulator/input_data/EuropeBoundary.shp"
gdf = gpd.read_file(shapefile_path)
bbox = gdf.total_bounds
bbox

array([-31.28903052,  34.93055094,  68.93136141,  81.85192337])

In [37]:
!ncinfo $inputData/$year/era5land/era5-land_total_precipitation_2015-01.nc

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    Conventions: CF-1.6
    history: 2023-06-20 04:05:16 GMT by grib_to_netcdf-2.25.1: /opt/ecmwf/mars-client/bin/grib_to_netcdf.bin -S param -o /cache/data3/adaptor.mars.internal-1687233692.3057544-27640-11-33b28015-7bc4-44ed-b4b9-99dcd6803491.nc /cache/tmp/33b28015-7bc4-44ed-b4b9-99dcd6803491-adaptor.mars.internal-1687233611.317501-27640-18-tmp.grib
    dimensions(sizes): longitude(3600), latitude(1801), time(744)
    variables(dimensions): float32 longitude(longitude), float32 latitude(latitude), int32 time(time), int16 tp(time, latitude, longitude)
    groups: 


In [8]:
%%time
### 0) read era5land data
# select Europe data of ERA5Land and save it as netcdf
all1 = xr.open_mfdataset(inputData+year+"/era5land/era5-land*.nc", parallel=True, chunks={"time":-1,"latitude":250,"longitude":250})
# all1 = xr.open_mfdataset(inputData+year+"/era5land/era5-land*.nc", parallel=True,chunks={"time":-1})
# temporal chunksize of open_mfdataset always depends on the single netcdf files
# spatial chunksize 200, less tasks but failed. chunksize 51, too many tasks. chunksize 100?
# all1 = xr.open_dataset(inputData+year+"/era5land/era5-land_10m_u_component_of_wind_2015-01.nc", chunks={"time":-1,"latitude":51,"longitude":51})
# all1=all1.chunk({"time":-1}) # the chunksize along time (8760 in total)? how much is optimal?

# lon_name = 'longitude'
# all1['longitude_adjusted'] = xr.where(
#     all1[lon_name] > 180,
#     all1[lon_name] - 360,
#     all1[lon_name])
# all1 = (
#     all1
#     .swap_dims({lon_name: 'longitude_adjusted'})
#     .sel(**{'longitude_adjusted': sorted(all1.longitude_adjusted)})
#     .drop(lon_name))
# all1 = all1.rename({'longitude_adjusted': lon_name})

all1['longitude'] = (all1['longitude'] + 180) % 360 - 180
all1 = all1.sortby('longitude')

all1 = all1.sel(
    latitude=slice(bbox[3], bbox[1]),  
    longitude=slice(bbox[0], bbox[2])  
)

CPU times: user 875 ms, sys: 85.4 ms, total: 960 ms
Wall time: 2.76 s


In [9]:
all1.chunks

Frozen({'time': (744, 672, 744, 720, 744, 720, 744, 744, 720, 744, 720, 744), 'latitude': (18, 100, 100, 100, 100, 51), 'longitude': (12, 100, 100, 100, 100, 100, 100, 100, 100, 100, 90)})

In [9]:
all1.nbytes/2**30

122.686006244272

In [18]:
all2 = all1.chunk({"time":500, "latitude":250, "longitude":250})

In [19]:
all2['sp']

<xarray.DataArray 'sp' (time: 8760, latitude: 469, longitude: 1002)>
dask.array<rechunk-merge, shape=(8760, 469, 1002), dtype=float32, chunksize=(5000, 100, 100), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float32 -31.2 -31.1 -31.0 -30.9 ... 68.7 68.8 68.9
  * latitude   (latitude) float32 81.8 81.7 81.6 81.5 ... 35.3 35.2 35.1 35.0
  * time       (time) datetime64[ns] 2015-01-01 ... 2015-12-31T23:00:00
Attributes:
    units:          Pa
    long_name:      Surface pressure
    standard_name:  surface_air_pressure

In [ ]:
%%time
# all2.to_netcdf('/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/1input_data/2015global/era5land/era5land2015_10km1027.nc',engine='h5netcdf') #, format='NETCDF4'
all2.to_zarr('/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/1input_data/2015global/era5land/era5land2015_10km1027.zarr',mode='w')


# test is the exported netcdf complete

In [13]:
lat1 = 40
lat2 = 60
lon1 = 2
lon2 = 22

In [20]:
test = xr.open_zarr(inputData+year+"/era5land/era5land2015_10km1027.zarr") 
# test = test.sel(
#     latitude=slice(lat2,lat1),  
#     longitude=slice(lon1, lon2)  
# )

In [24]:
test['sp']

<xarray.DataArray 'sp' (time: 8760, latitude: 469, longitude: 1002)>
dask.array<open_dataset-005fb76cb4774064a0be85da8d70d98asp, shape=(8760, 469, 1002), dtype=float32, chunksize=(500, 250, 250), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float32 81.8 81.7 81.6 81.5 ... 35.3 35.2 35.1 35.0
  * longitude  (longitude) float32 -31.2 -31.1 -31.0 -30.9 ... 68.7 68.8 68.9
  * time       (time) datetime64[ns] 2015-01-01 ... 2015-12-31T23:00:00
Attributes:
    long_name:      Surface pressure
    standard_name:  surface_air_pressure
    units:          Pa

In [30]:
test['u10'].mean(dim={"longitude","latitude"}).isnull().sum().compute()

<xarray.DataArray 'u10' ()>
array(0)

In [22]:
a = np.nanmean(test['u10'], axis=(1,2)) #xarray
nan_mask = np.isnan(a)

# use np.sum() to count the number of True, which is NaN
nan_count = np.sum(nan_mask)

In [23]:
nan_count

0